# Split knowns by confidence

In [1]:
import pandas as pd
import os
import json
import numpy as np

## Load the data
With metadata.

In [11]:
data = pd.read_json("/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/counterfact_check/preds_wiki_nb_pb.jsonl", lines=True)
# only keep the top pred (same as Meng et al.)
data = data[data.rank_answers<1].reset_index(drop=True).iloc[:1000]
data

,known_id,sub_label,obj_label,template,prediction,prompt,predicate_id,sub_view_rates,obj_view_rates,answers,p_answers,rank_answers,string_match,person_name,used_template,prompt_bias
0,0,Vinson Massif,Antarctica,{} is located in the continent,of Antarctica. It is the largest of the three,Vinson Massif is located in the continent of,P30,13620.000000,161094.333333,Antarctica,0.094846,0,False,False,[X] is located in the continent of,False
1,1,Beats Music,Apple,{} is owned by,"Apple, which is also the owner of Beats Elect...",Beats Music is owned by,P127,4766.166667,109751.166667,Apple,0.683708,0,False,False,[X] is owned by,False
2,2,Audible.com,Amazon,{} is owned by,"Amazon.com, Inc. or its affiliates.",Audible.com is owned by,P127,1840.583333,28795.916667,Amazon,0.255502,0,False,False,[X] is owned by,False
3,3,The Big Bang Theory,CBS,{} premieres on,CBS on September 22.<|endoftext|>,The Big Bang Theory premieres on,P449,441020.666667,5826.750000,CBS,0.388209,0,False,False,[X] premieres on,False
4,4,MacApp,Apple,"{}, a product created by",Apple to help developers create apps for the ...,"MacApp, a product created by",P178,787.666667,109751.166667,Apple,0.127723,0,False,False,"[X], a product created by",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Edvard Grieg,piano,"{}, performing on the",piano.\n\nThe music of Grieg is,"Edvard Grieg, performing on the",P1303,23086.000000,67049.916667,piano,0.032681,0,False,False,"[X], performing on the",True
996,996,Paul Klebnikov,Russian,The native language of {} is,Russian.\n\nContents show]\n\nBi,The native language of Paul Klebnikov is,P103,1276.583333,8843.833333,Russian,0.448957,0,False,True,The native language of [X] is,False
997,997,Mojang,Microsoft,{} is owned by,"Microsoft, which is a major sponsor of the game",Mojang is owned by,P127,25075.750000,267479.166667,Microsoft,0.368855,0,False,False,[X] is owned by,True
998,998,Giovanni di Bicci de' Medici,Florence,{} died in the city of,Florence in 1492. He was the son of,Giovanni di Bicci de' Medici died in the city of,P20,14974.333333,87461.500000,Florence,0.169127,0,False,True,[X] died in the city of,False


In [12]:
data[data.p_answers<0.2]

,known_id,sub_label,obj_label,template,prediction,prompt,predicate_id,sub_view_rates,obj_view_rates,answers,p_answers,rank_answers,string_match,person_name,used_template,prompt_bias
0,0,Vinson Massif,Antarctica,{} is located in the continent,of Antarctica. It is the largest of the three,Vinson Massif is located in the continent of,P30,13620.000000,161094.333333,Antarctica,0.094846,0,False,False,[X] is located in the continent of,False
4,4,MacApp,Apple,"{}, a product created by",Apple to help developers create apps for the ...,"MacApp, a product created by",P178,787.666667,109751.166667,Apple,0.127723,0,False,False,"[X], a product created by",True
9,9,De finibus bonorum et malorum,Latin,The original language of {} is,"the same as the Latin of the Roman law,",The original language of De finibus bonorum et...,P364,4112.083333,139995.916667,Latin,0.054921,0,False,False,The original language of [X] is the same as the,False
12,12,Il Gazzettino,Italian,{} was written in,"the early 1980s, when the Italian government was","Il Gazzettino was written in the early 1980s, ...",P407,1104.083333,6688.916667,Italian,0.110233,0,False,False,"[X] was written in the early 1980s, when the",False
13,13,Xamarin,Microsoft,"{}, from","the Microsoft Azure team, has announced that ...","Xamarin, from the",P127,17389.500000,267479.166667,Microsoft,0.054699,0,False,False,"[X], from the",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,989,Fatimid caliphate,Cairo,The capital of {} is,the city of Cairo. The city is the largest,The capital of Fatimid caliphate is the city of,P36,371.166667,67103.000000,Cairo,0.074772,0,False,False,The capital of [X] is the city of,False
990,990,"Claude Henri de Rouvroy, comte de Saint-Simon",French,The language used by {} is,a good example of the way in which the French,"The language used by Claude Henri de Rouvroy, ...",P1412,1801.166667,10622.750000,French,0.082305,0,False,True,The language used by [X] is a good example of ...,False
994,994,Basil of Caesarea,bishop,{} holds the title of,the first bishop of Jerusalem. He was the son,Basil of Caesarea holds the title of the first,P39,9738.333333,26032.583333,bishop,0.159965,0,False,False,[X] holds the title of the first,False
995,995,Edvard Grieg,piano,"{}, performing on the",piano.\n\nThe music of Grieg is,"Edvard Grieg, performing on the",P1303,23086.000000,67049.916667,piano,0.032681,0,False,False,"[X], performing on the",True


In [13]:
data[data.p_answers>=0.2]

,known_id,sub_label,obj_label,template,prediction,prompt,predicate_id,sub_view_rates,obj_view_rates,answers,p_answers,rank_answers,string_match,person_name,used_template,prompt_bias
1,1,Beats Music,Apple,{} is owned by,"Apple, which is also the owner of Beats Elect...",Beats Music is owned by,P127,4766.166667,109751.166667,Apple,0.683708,0,False,False,[X] is owned by,False
2,2,Audible.com,Amazon,{} is owned by,"Amazon.com, Inc. or its affiliates.",Audible.com is owned by,P127,1840.583333,28795.916667,Amazon,0.255502,0,False,False,[X] is owned by,False
3,3,The Big Bang Theory,CBS,{} premieres on,CBS on September 22.<|endoftext|>,The Big Bang Theory premieres on,P449,441020.666667,5826.750000,CBS,0.388209,0,False,False,[X] premieres on,False
5,5,Giuseppe Angeli,Italy,"{}, who has a citizenship of","Italy, was arrested in the US in May.","Giuseppe Angeli, who has a citizenship of",P27,182.666667,338858.666667,Italy,0.279786,0,False,True,"[X], who has a citizenship of",False
6,6,Catalonia,Europe,{} belongs to the continent of,"Europe, and the Spanish government has been t...",Catalonia belongs to the continent of,P30,96646.000000,295477.000000,Europe,0.877088,0,False,False,[X] belongs to the continent of,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,992,Yuki Uchida,Japan,{} is a citizen of,Japan.\n\nThe following is a list of,Yuki Uchida is a citizen of,P27,2154.666667,491464.833333,Japan,0.501128,0,False,True,[X] is a citizen of,False
993,993,Dawid Janowski,Poland,{} is a citizen of,Poland and a graduate of the University of Ch...,Dawid Janowski is a citizen of,P27,504.416667,277818.750000,Poland,0.356440,0,False,True,[X] is a citizen of,False
996,996,Paul Klebnikov,Russian,The native language of {} is,Russian.\n\nContents show]\n\nBi,The native language of Paul Klebnikov is,P103,1276.583333,8843.833333,Russian,0.448957,0,False,True,The native language of [X] is,False
997,997,Mojang,Microsoft,{} is owned by,"Microsoft, which is a major sponsor of the game",Mojang is owned by,P127,25075.750000,267479.166667,Microsoft,0.368855,0,False,False,[X] is owned by,True


Save the data splits

In [14]:
data[data.p_answers<0.2].to_json("/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_knowns_low_confidence.json", orient="records")
data[data.p_answers>=0.2].to_json("/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_knowns_high_confidence.json", orient="records")